In [9]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
# Set random seed for reproducibility
np.random.seed(42)

# --- NeuralNetwork Class (for Regression - with mini-batching in mind) ---
class NeuralNetwork:
    def __init__(self, layer_sizes, learning_rate=0.0001): # Re-evaluating learning rate
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.weights = []
        self.biases = []
        self.num_layers = len(self.layer_sizes) - 1
        self.initialize_network()

    def initialize_network(self):
        for i in range(self.num_layers):
            input_dim = self.layer_sizes[i]
            output_dim = self.layer_sizes[i+1]
            scale_w = np.sqrt(2.0 / (input_dim + output_dim))
            self.weights.append(np.random.normal(loc=0, scale=scale_w, size=(input_dim, output_dim)))
            self.biases.append(np.random.normal(loc=0, scale=0.01, size=(1, output_dim)))
        self.n_params = sum(w.size for w in self.weights) + sum(b.size for b in self.biases)

    def sigmoid(self, X):
        X = np.clip(X, -50, 50) # Clip for numerical stability
        return 1 / (1 + np.exp(-X))

    def forward_pass(self, X):
        self.activations = [X]
        self.zs = []
        for i in range(self.num_layers):
            current_input = self.activations[-1]
            Z = np.dot(current_input, self.weights[i]) + self.biases[i]
            self.zs.append(Z)
            A = self.sigmoid(Z) if i < self.num_layers - 1 else Z
            self.activations.append(A)
        return self.activations[-1]

    def backward_pass(self, X, Y):
        output_predictions = self.activations[-1]
        delta = output_predictions - Y

        deltas = [delta]
        for i in range(self.num_layers - 1, 0, -1):
            W_l = self.weights[i]
            Z_prev = self.zs[i-1]
            delta = (deltas[-1] @ W_l.T) * (self.sigmoid(Z_prev) * (1 - self.sigmoid(Z_prev)))
            deltas.append(delta)
        deltas.reverse()

        for i in range(self.num_layers):
            weights_delta = np.dot(self.activations[i].T, deltas[i])
            biases_delta = np.sum(deltas[i], axis=0, keepdims=True)

            if not np.all(np.isfinite(weights_delta)): weights_delta = np.zeros_like(weights_delta)
            if not np.all(np.isfinite(biases_delta)): biases_delta = np.zeros_like(biases_delta)

            self.weights[i] -= self.learning_rate * weights_delta
            self.biases[i] -= self.learning_rate * biases_delta

    def encode(self):
        all_params = []
        for w in self.weights: all_params.append(w.ravel())
        for b in self.biases: all_params.append(b.ravel())
        return np.concatenate(all_params)

    def decode(self, theta):
        decoded_weights = []
        decoded_biases = []
        offset = 0
        for i in range(self.num_layers):
            input_dim = self.layer_sizes[i]
            output_dim = self.layer_sizes[i+1]
            weight_size = input_dim * output_dim
            decoded_weights.append(theta[offset : offset + weight_size].reshape(input_dim, output_dim))
            offset += weight_size
            bias_size = output_dim
            decoded_biases.append(theta[offset : offset + bias_size].reshape(1, output_dim))
            offset += bias_size
        self.weights = decoded_weights
        self.biases = decoded_biases

    def evaluate_proposal(self, theta, X_data):
        temp_weights = []
        temp_biases = []
        offset = 0
        for i in range(self.num_layers):
            input_dim = self.layer_sizes[i]
            output_dim = self.layer_sizes[i+1]
            weight_size = input_dim * output_dim
            temp_weights.append(theta[offset : offset + weight_size].reshape(input_dim, output_dim))
            offset += weight_size
            bias_size = output_dim
            temp_biases.append(theta[offset : offset + bias_size].reshape(1, output_dim))
            offset += bias_size

        activations = [X_data]
        for i in range(self.num_layers - 1):
            Z = np.dot(activations[-1], temp_weights[i]) + temp_biases[i]
            activations.append(self.sigmoid(Z))
        fx = np.dot(activations[-1], temp_weights[-1]) + temp_biases[-1]
        return fx

    # Modified langevin_gradient for mini-batching
    def langevin_gradient(self, x_data, y_data, theta, depth, batch_size): # Added batch_size
        original_weights = [w.copy() for w in self.weights]
        original_biases = [b.copy() for b in self.biases]

        self.decode(theta)

        num_samples = x_data.shape[0]
        indices = np.arange(num_samples)

        for _ in range(0, depth):
            # Mini-batching
            np.random.shuffle(indices)
            for start_idx in range(0, num_samples, batch_size):
                end_idx = min(start_idx + batch_size, num_samples)
                batch_indices = indices[start_idx:end_idx]
                
                x_batch = x_data[batch_indices]
                y_batch = y_data[batch_indices]

                self.forward_pass(x_batch)
                self.backward_pass(x_batch, y_batch)

        theta_updated = self.encode()

        self.weights = original_weights
        self.biases = original_biases

        return theta_updated


In [11]:
# --- MCMC Class ---
class MCMC:
    def __init__(self, n_samples, n_burnin, x_data, y_data, x_test, y_test, layer_sizes, 
                 noise_variance=0.1, batch_size=32): # Added batch_size to MCMC __init__
        self.n_samples = n_samples
        self.n_burnin = n_burnin
        self.x_data = x_data
        self.y_data = y_data
        self.x_test = x_test
        self.y_test = y_test

        self.step_theta = 5e-7 # Even smaller MCMC step size might be needed, tune carefully!
        self.sigma_squared_prior = 10.0
        self.noise_variance = noise_variance

        self.model = NeuralNetwork(layer_sizes)
        self.theta_size = self.model.n_params

        self.use_langevin_gradients = True
        self.sgd_depth = 1 # Number of mini-batch updates within one Langevin step
        self.l_prob = 0.9 # Higher probability of using Langevin gradient
        self.batch_size = batch_size # Mini-batch size for Langevin gradients

        self.pos_theta = None
        self.pred_y = None
        self.rmse_data = None
        self.r2_data = None
        self.test_pred_y = None
        self.test_rmse_data = None
        self.test_r2_data = None

    @staticmethod
    def evaluate_metrics(predictions, targets):
        if not np.all(np.isfinite(predictions)) or not np.all(np.isfinite(targets)):
            return np.inf, -np.inf
        rmse = np.sqrt(mean_squared_error(targets, predictions))
        r2 = r2_score(targets, predictions)
        return rmse, r2

    def likelihood_function(self, theta, test=False):
        x_data_eval = self.x_test if test else self.x_data
        y_data_eval = self.y_test if test else self.y_data

        model_prediction = self.model.evaluate_proposal(theta, x_data_eval)

        if not np.all(np.isfinite(model_prediction)):
            return -np.inf, model_prediction, np.inf, -np.inf

        N = y_data_eval.shape[0]
        sum_sq_error = np.sum(np.square(y_data_eval - model_prediction))

        if self.noise_variance <= 1e-9:
            return -np.inf, model_prediction, np.inf, -np.inf

        log_likelihood = -N/2.0 * np.log(2 * np.pi * self.noise_variance) - (1/(2 * self.noise_variance)) * sum_sq_error
        
        rmse, r2 = self.evaluate_metrics(model_prediction, y_data_eval)

        return log_likelihood, model_prediction, rmse, r2

    def prior(self, sigma_squared_prior, theta):
        if sigma_squared_prior <= 1e-9:
            return -np.inf
        part1 = -self.model.n_params / 2.0 * np.log(sigma_squared_prior)
        part2 = 1 / (2.0 * sigma_squared_prior) * (np.sum(np.square(theta)))
        return part1 - part2

    def MCMC_sampler(self):
        output_dim = self.model.layer_sizes[-1]
        pos_theta = np.zeros((self.n_samples, self.theta_size))
        pred_y = np.zeros((self.n_samples, self.x_data.shape[0], output_dim))
        test_pred_y = np.zeros((self.n_samples, self.x_test.shape[0], output_dim))
        rmse_data = np.zeros(self.n_samples)
        r2_data = np.zeros(self.n_samples)
        test_rmse_data = np.zeros(self.n_samples)
        test_r2_data = np.zeros(self.n_samples)

        theta = np.random.randn(self.theta_size) * 0.01 # Smaller initial scale

        prior_val = self.prior(self.sigma_squared_prior, theta)
        (likelihood, initial_train_pred, initial_train_rmse, initial_train_r2) = self.likelihood_function(theta, test=False)
        (test_likelihood, initial_test_pred, initial_test_rmse, initial_test_r2) = self.likelihood_function(theta, test=True)

        if not np.isfinite(likelihood) or not np.isfinite(prior_val):
            print("Warning: Initial likelihood or prior is non-finite. Re-initializing theta to a smaller scale.")
            theta = np.random.randn(self.theta_size) * 0.001
            prior_val = self.prior(self.sigma_squared_prior, theta)
            (likelihood, initial_train_pred, initial_train_rmse, initial_train_r2) = self.likelihood_function(theta, test=False)
            (test_likelihood, initial_test_pred, initial_test_rmse, initial_test_r2) = self.likelihood_function(theta, test=True)
            if not np.isfinite(likelihood) or not np.isfinite(prior_val):
                 print("Critical: Initial likelihood/prior still non-finite after re-init. MCMC chain may struggle.")

        pos_theta[0, :] = theta
        pred_y[0, :, :] = initial_train_pred
        rmse_data[0] = initial_train_rmse
        r2_data[0] = initial_train_r2
        test_pred_y[0, :, :] = initial_test_pred
        test_rmse_data[0] = initial_test_rmse
        test_r2_data[0] = initial_test_r2

        n_accepted_samples = 0

        print(f"MCMC Chain started for {self.n_samples} samples with batch_size={self.batch_size}...")
        print(f"Initial Train RMSE: {initial_train_rmse:.4f}, R2: {initial_train_r2:.4f}")
        print(f"Initial Test RMSE: {initial_test_rmse:.4f}, R2: {initial_test_r2:.4f}")

        for ii in range(1, self.n_samples):
            theta_current = pos_theta[ii - 1, :]
            prior_current = prior_val
            likelihood_current = likelihood

            lx = np.random.uniform(0, 1)
            if lx < self.l_prob and self.use_langevin_gradients:
                # Pass batch_size to langevin_gradient
                theta_gd = self.model.langevin_gradient(self.x_data, self.y_data, theta_current, self.sgd_depth, self.batch_size)
                theta_proposal = np.random.normal(theta_gd, self.step_theta, self.theta_size)
            else:
                theta_proposal = np.random.normal(theta_current, self.step_theta, self.theta_size)

            prior_proposal = self.prior(self.sigma_squared_prior, theta_proposal)
            (likelihood_proposal, current_train_pred, current_train_rmse, current_train_r2) = self.likelihood_function(theta_proposal, test=False)
            (test_likelihood_proposal, current_test_pred, current_test_rmse, current_test_r2) = self.likelihood_function(theta_proposal, test=True)

            if not np.isfinite(likelihood_proposal) or not np.isfinite(prior_proposal):
                mh_prob = 0.0
            else:
                diff_likelihood = likelihood_proposal - likelihood_current
                diff_prior = prior_proposal - prior_current

                diff_prop = 0.0
                if lx < self.l_prob and self.use_langevin_gradients:
                    # Pass batch_size to langevin_gradient for reverse step
                    theta_gd_reverse = self.model.langevin_gradient(self.x_data, self.y_data, theta_proposal, self.sgd_depth, self.batch_size)
                    log_q_prop_given_curr = -0.5 * np.sum(np.square(theta_proposal - theta_gd)) / (self.step_theta**2)
                    log_q_curr_given_prop = -0.5 * np.sum(np.square(theta_current - theta_gd_reverse)) / (self.step_theta**2)
                    diff_prop = log_q_curr_given_prop - log_q_prop_given_curr
                    if not np.isfinite(diff_prop): mh_prob = 0.0

                if np.isfinite(diff_likelihood) and np.isfinite(diff_prior) and np.isfinite(diff_prop):
                    mh_prob = np.min([1.0, np.exp(diff_likelihood + diff_prior + diff_prop)])
                else:
                    mh_prob = 0.0

            u = np.random.uniform(0, 1)

            if u < mh_prob:
                pos_theta[ii, :] = theta_proposal
                likelihood = likelihood_proposal
                prior_val = prior_proposal
                pred_y[ii, :, :] = current_train_pred
                rmse_data[ii] = current_train_rmse
                r2_data[ii] = current_train_r2
                test_pred_y[ii, :, :] = current_test_pred
                test_rmse_data[ii] = current_test_rmse
                test_r2_data[ii] = current_test_r2
                n_accepted_samples += 1
            else:
                pos_theta[ii, :] = pos_theta[ii - 1, :]
                pred_y[ii, :, :] = pred_y[ii - 1, :, :]
                rmse_data[ii] = rmse_data[ii - 1]
                r2_data[ii] = r2_data[ii - 1]
                test_pred_y[ii, :, :] = test_pred_y[ii - 1, :, :]
                test_rmse_data[ii] = test_rmse_data[ii - 1]
                test_r2_data[ii] = test_r2_data[ii - 1]

            if (ii + 1) % 500 == 0:
                acceptance_rate = (n_accepted_samples / (ii + 1)) * 100
                print(f"Sample {ii+1}/{self.n_samples} | Accept Rate: {acceptance_rate:.2f}% | "
                      f"Train RMSE: {rmse_data[ii]:.4f}, R2: {r2_data[ii]:.4f} | "
                      f"Test RMSE: {test_rmse_data[ii]:.4f}, R2: {test_r2_data[ii]:.4f}")

        print("MCMC sampling complete. Applying burn-in.")
        
        self.pos_theta = pos_theta[self.n_burnin:, :]
        self.pred_y = pred_y[self.n_burnin:, :, :]
        self.rmse_data = rmse_data[self.n_burnin:]
        self.r2_data = r2_data[self.n_burnin:]
        self.test_pred_y = test_pred_y[self.n_burnin:, :, :]
        self.test_rmse_data = test_rmse_data[self.n_burnin:]
        self.test_r2_data = test_r2_data[self.n_burnin:]

        results_df = pd.DataFrame(self.pos_theta, columns=[f"theta_{i}" for i in range(self.theta_size)])
        return results_df

In [12]:
# --- Data Preparation and Execution for Regression ---

def load_and_preprocess_regression_data(dataset_loader, test_size=0.2, random_state=42):
    data = dataset_loader()
    X, y = data.data, data.target.reshape(-1, 1)

    scaler_X = StandardScaler()
    X = scaler_X.fit_transform(X)

    scaler_y = StandardScaler()
    y = scaler_y.fit_transform(y) # Scale target variable too!

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train, y_test, scaler_X, scaler_y


In [13]:
# --- Run Regression Example ---
print("\n" + "="*50)
print("Testing BNN for Regression on California Housing Dataset (Faster NumPy Version)")
print("="*50)

X_train_reg, X_test_reg, y_train_reg, y_test_reg, scaler_X_reg, scaler_y_reg = load_and_preprocess_regression_data(fetch_california_housing)

input_dim_reg = X_train_reg.shape[1]
output_dim_reg = y_train_reg.shape[1]
hidden_layer_nodes_reg = [50, 25]
layer_sizes_reg = [input_dim_reg] + hidden_layer_nodes_reg + [output_dim_reg]

print(f"Network Architecture for Regression: {layer_sizes_reg}")



Testing BNN for Regression on California Housing Dataset (Faster NumPy Version)
Network Architecture for Regression: [8, 50, 25, 1]


In [ ]:
# MCMC parameters (tuned for faster NumPy execution and more stability)
n_samples_reg = 50000 # Keep samples high for good posterior estimation
n_burnin_reg = 10000
noise_variance_reg = 0.1
batch_size_reg = 128 # Mini-batch size for Langevin gradients (CRITICAL for speed)

print("Starting MCMC sampling for Regression dataset...")
bnn_mcmc_reg = MCMC(n_samples_reg, n_burnin_reg, 
                    X_train_reg, y_train_reg, 
                    X_test_reg, y_test_reg, 
                    layer_sizes_reg, 
                    noise_variance=noise_variance_reg,
                    batch_size=batch_size_reg)
posterior_samples_df_reg = bnn_mcmc_reg.MCMC_sampler() 
print("MCMC sampling for Regression dataset finished.")

print("\nShape of posterior samples (after burn-in):", posterior_samples_df_reg.shape)
print("First 5 posterior samples:\n", posterior_samples_df_reg.head())


Starting MCMC sampling for Regression dataset...
MCMC Chain started for 50000 samples with batch_size=128...
Initial Train RMSE: 1.0020, R2: -0.0001
Initial Test RMSE: 0.9920, R2: -0.0000


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_32956\2810185149.py:140: RuntimeWarning: overflow encountered in exp
  mh_prob = np.min([1.0, np.exp(diff_likelihood + diff_prior + diff_prop)])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_32956\2810185149.py:140: RuntimeWarning: overflow encountered in exp
  mh_prob = np.min([1.0, np.exp(diff_likelihood + diff_prior + diff_prop)])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_32956\2810185149.py:140: RuntimeWarning: overflow encountered in exp
  mh_prob = np.min([1.0, np.exp(diff_likelihood + diff_prior + diff_prop)])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_32956\2810185149.py:140: RuntimeWarning: overflow encountered in exp
  mh_prob = np.min([1.0, np.exp(diff_likelihood + diff_prior + diff_prop)])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_32956\2810185149.py:140: RuntimeWarning: overflow encountered in exp
  mh_prob = np.min([1.0, np.exp(diff_likelihood + diff_prior + diff_prop)])


Sample 500/50000 | Accept Rate: 7.00% | Train RMSE: 0.5839, R2: 0.6604 | Test RMSE: 0.5968, R2: 0.6381
Sample 1000/50000 | Accept Rate: 9.60% | Train RMSE: 0.5427, R2: 0.7066 | Test RMSE: 0.5525, R2: 0.6898
Sample 1500/50000 | Accept Rate: 9.93% | Train RMSE: 0.5235, R2: 0.7271 | Test RMSE: 0.5326, R2: 0.7118
Sample 2000/50000 | Accept Rate: 9.30% | Train RMSE: 0.5144, R2: 0.7364 | Test RMSE: 0.5238, R2: 0.7212
Sample 2500/50000 | Accept Rate: 8.40% | Train RMSE: 0.5092, R2: 0.7417 | Test RMSE: 0.5191, R2: 0.7262
Sample 3000/50000 | Accept Rate: 7.80% | Train RMSE: 0.5006, R2: 0.7504 | Test RMSE: 0.5106, R2: 0.7351
Sample 3500/50000 | Accept Rate: 7.49% | Train RMSE: 0.4896, R2: 0.7613 | Test RMSE: 0.4992, R2: 0.7468
Sample 4000/50000 | Accept Rate: 7.35% | Train RMSE: 0.4827, R2: 0.7679 | Test RMSE: 0.4923, R2: 0.7537
Sample 4500/50000 | Accept Rate: 7.44% | Train RMSE: 0.4782, R2: 0.7723 | Test RMSE: 0.4876, R2: 0.7584


In [ ]:
# --- Evaluate Performance for Regression Dataset ---
print("\n--- Final Performance Evaluation for Regression Dataset ---")
    
mean_train_predictions_reg_scaled = np.mean(bnn_mcmc_reg.pred_y, axis=0)
mean_test_predictions_reg_scaled = np.mean(bnn_mcmc_reg.test_pred_y, axis=0)

final_train_predictions_reg = scaler_y_reg.inverse_transform(mean_train_predictions_reg_scaled)
final_test_predictions_reg = scaler_y_reg.inverse_transform(mean_test_predictions_reg_scaled)

original_y_train_reg = scaler_y_reg.inverse_transform(y_train_reg)
original_y_test_reg = scaler_y_reg.inverse_transform(y_test_reg)

In [ ]:
final_train_rmse, final_train_r2 = mean_squared_error(original_y_train_reg, final_train_predictions_reg), r2_score(original_y_train_reg, final_train_predictions_reg)
final_test_rmse, final_test_r2 = mean_squared_error(original_y_test_reg, final_test_predictions_reg), r2_score(original_y_test_reg, final_test_predictions_reg)

print(f"\nFinal Train RMSE (from posterior predictive mean): {final_train_rmse:.4f}")
print(f"Final Train R2 (from posterior predictive mean): {final_train_r2:.4f}")
print(f"Final Test RMSE (from posterior predictive mean): {final_test_rmse:.4f}")
print(f"Final Test R2 (from posterior predictive mean): {final_test_r2:.4f}")

avg_sampled_train_rmse = np.mean(bnn_mcmc_reg.rmse_data[np.isfinite(bnn_mcmc_reg.rmse_data)])
avg_sampled_train_r2 = np.mean(bnn_mcmc_reg.r2_data[np.isfinite(bnn_mcmc_reg.r2_data)])
avg_sampled_test_rmse = np.mean(bnn_mcmc_reg.test_rmse_data[np.isfinite(bnn_mcmc_reg.test_rmse_data)])
avg_sampled_test_r2 = np.mean(bnn_mcmc_reg.test_r2_data[np.isfinite(bnn_mcmc_reg.test_r2_data)])

print(f"\nAverage Train RMSE (across accepted finite samples): {avg_sampled_train_rmse:.4f}")
print(f"Average Train R2 (across accepted finite samples): {avg_sampled_train_r2:.4f}")
print(f"Average Test RMSE (across accepted finite samples): {avg_sampled_test_rmse:.4f}")
print(f"Average Test R2 (across accepted finite samples): {avg_sampled_test_r2:.4f}")

print("\n" + "="*50)
print("End of Faster NumPy Regression BNN Test")
print("="*50)

## SKLEARN RESULTS

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score

# Set random seed for reproducibility
np.random.seed(42)

# --- Data Loading and Preprocessing 
def load_and_preprocess_regression_data(dataset_loader, test_size=0.2, random_state=42):
    data = dataset_loader()
    X, y = data.data, data.target.reshape(-1, 1)

    scaler_X = StandardScaler()
    X = scaler_X.fit_transform(X)

    scaler_y = StandardScaler()
    y = scaler_y.fit_transform(y) 

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train, y_test, scaler_X, scaler_y

In [ ]:
# --- Load and Prepare California Housing Data ---
print("--- Preparing California Housing Data ---")
X_train_reg, X_test_reg, y_train_reg, y_test_reg, scaler_X_reg, scaler_y_reg = load_and_preprocess_regression_data(fetch_california_housing)

print(f"Training features shape: {X_train_reg.shape}")
print(f"Training target shape: {y_train_reg.shape}")
print(f"Test features shape: {X_test_reg.shape}")
print(f"Test target shape: {y_test_reg.shape}")

--- Preparing California Housing Data ---
Training features shape: (16512, 8)
Training target shape: (16512, 1)
Test features shape: (4128, 8)
Test target shape: (4128, 1)


In [ ]:
# --- Train a Simple Linear Regression Model ---
print("\n--- Training Linear Regression Model ---")

# Create the model
linear_model = LinearRegression()

# Train the model using the scaled training data
linear_model.fit(X_train_reg, y_train_reg)

# --- Make Predictions ---
# Predict on the scaled training data
train_predictions_scaled = linear_model.predict(X_train_reg)
# Predict on the scaled test data
test_predictions_scaled = linear_model.predict(X_test_reg)


--- Training Linear Regression Model ---


In [ ]:
# --- Evaluate Performance (Inverse transform to original scale for metrics) ---
print("\n--- Evaluating Linear Regression Performance ---")

# Inverse transform predictions and true values to original scale
final_train_predictions = scaler_y_reg.inverse_transform(train_predictions_scaled)
final_test_predictions = scaler_y_reg.inverse_transform(test_predictions_scaled)

original_y_train = scaler_y_reg.inverse_transform(y_train_reg)
original_y_test = scaler_y_reg.inverse_transform(y_test_reg)

# Calculate RMSE and R2 for training set
train_rmse = mean_squared_error(original_y_train, final_train_predictions)
train_r2 = r2_score(original_y_train, final_train_predictions)

# Calculate RMSE and R2 for test set
test_rmse = mean_squared_error(original_y_test, final_test_predictions)
test_r2 = r2_score(original_y_test, final_test_predictions)

print(f"Train RMSE: {train_rmse:.4f}")
print(f"Train R2 Score: {train_r2:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test R2 Score: {test_r2:.4f}")

print("\n--- Linear Regression Performance Evaluation Complete ---")


--- Evaluating Linear Regression Performance ---
Train RMSE: 0.5179
Train R2 Score: 0.6126
Test RMSE: 0.5559
Test R2 Score: 0.5758

--- Linear Regression Performance Evaluation Complete ---
